<a href="https://colab.research.google.com/github/zih-yan/abc1/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q pyngrok flask flask_ngrok
!pip install -q flask line-bot-sdk

In [ ]:

# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
    ngrok.connect(5000)
    run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage
)


app = Flask(__name__)
run_with_ngrok(app)

line_bot_api = LineBotApi(os.getenv("LINE_CHANNEL_ACCESS_TOKEN"))
handler = WebhookHandler(os.getenv("LINE_CHANNEL_SECRET"))

@app.route("/callback", methods=["POST"])
def callback():
    signature = request.headers["X-Line-Signature"]
    body = request.get_data(as_text=True)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return "OK"

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    msg = event.message.text
    reply = ""

    if msg == "確定模板":
        reply = TemplateSendMessage(
            alt_text="這是確定樣板",
            template=ConfirmTemplate(
                text="你確定要吃晚餐了嗎？",
                actions=[
                    MessageAction(label="是", text="我吃飽了"),
                    MessageAction(label="否", text="我還不餓，再等等")
                ]
            )
        )
    elif msg == "carousel":
        carousel_template = CarouselTemplate(
            columns=[
                CarouselColumn(
                    thumbnail_image_url="https://upload.wikimedia.org/wikipedia/zh/7/7d/Summer_Wars_poster.jpg",
                    title="夏日大作戰",
                    text="這部動畫超好看～",
                    actions=[
                        URIAction(label="點我看維基百科", uri="https://zh.wikipedia.org/zh-tw/%E5%A4%8F%E6%97%A5%E5%A4%A7%E4%BD%9C%E6%88%A0"),
                        MessageAction(label="我想看這部", text="我要看夏日大作戰～")
                    ]
                ),
                CarouselColumn(
                    thumbnail_image_url="https://upload.wikimedia.org/wikipedia/zh/thumb/4/4f/Castle_of_Cagliostro_poster.png/250px-Castle_of_Cagliostro_poster.png",
                    title="魯邦三世：卡里奧斯特羅城",
                    text="這部也不錯喔～",
                    actions=[
                        URIAction(label="點我看維基百科", uri="https://zh.wikipedia.org/zh-tw/%E9%AD%AF%E9%82%A6%E4%B8%89%E4%B8%96%EF%BC%9A%E5%8D%A1%E9%87%8C%E5%A5%A7%E6%96%AF%E7%89%B9%E7%BE%85%E5%9F%8E"),
                        MessageAction(label="我也想看", text="我也想看魯邦三世 卡里奧斯特羅城～")
                    ]
                )
            ]
        )

        reply = TemplateSendMessage(
            alt_text="這是轉盤樣板",
            template=carousel_template
        )
    else:
        reply = TextSendMessage(text="Thanks!")

    line_bot_api.reply_message(
        event.reply_token,
        reply
    )

# for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


<ipython-input-11-1ecf35b604f0>:25: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(os.getenv("LINE_CHANNEL_ACCESS_TOKEN"))
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connect